In [1]:
import tensorflow as tf

In [2]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [3]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [4]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-5.3670716e-01,  1.8872678e-01,  1.7133373e-01, -7.3884130e-02,
          4.9968958e-02, -4.7088027e-01,  2.1135634e-01, -3.8023472e-02,
          1.1651206e-01, -2.3787758e-01],
        [ 2.0208943e-01,  4.7484750e-01, -5.9792042e-01,  2.5456846e-02,
          1.4027894e-02, -2.3020181e-01,  1.0987747e-01, -4.6343809e-01,
         -5.2723247e-01, -6.2519521e-01],
        [ 2.9263633e-01, -6.3150990e-01, -5.3799534e-01,  1.6969436e-01,
          1.7870492e-01, -5.6019378e-01, -4.3439096e-01, -4.9516881e-01,
         -2.5274494e-01,  4.1244286e-01],
        [ 4.3834931e-01, -3.9197171e-01,  4.9543566e-01, -2.0698315e-01,
         -5.4886073e-01, -9.7492695e-02,  1.1797130e-02,  4.9783248e-01,
         -5.1575345e-01, -3.8776937e-01],
        [-4.9638748e-04, -3.0576208e-01, -5.8330339e-01, -5.6744248e-01,
          6.5116167e-02,  7.0884764e-02,  2.7545333e-01, -6.0635167e-01,
         -2.1278188e-01,  4.15110

In [6]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-5.3670716e-01,  1.8872678e-01,  1.7133373e-01, -7.3884130e-02,
          4.9968958e-02, -4.7088027e-01,  2.1135634e-01, -3.8023472e-02,
          1.1651206e-01, -2.3787758e-01],
        [ 2.0208943e-01,  4.7484750e-01, -5.9792042e-01,  2.5456846e-02,
          1.4027894e-02, -2.3020181e-01,  1.0987747e-01, -4.6343809e-01,
         -5.2723247e-01, -6.2519521e-01],
        [ 2.9263633e-01, -6.3150990e-01, -5.3799534e-01,  1.6969436e-01,
          1.7870492e-01, -5.6019378e-01, -4.3439096e-01, -4.9516881e-01,
         -2.5274494e-01,  4.1244286e-01],
        [ 4.3834931e-01, -3.9197171e-01,  4.9543566e-01, -2.0698315e-01,
         -5.4886073e-01, -9.7492695e-02,  1.1797130e-02,  4.9783248e-01,
         -5.1575345e-01, -3.8776937e-01],
        [-4.9638748e-04, -3.0576208e-01, -5.8330339e-01, -5.6744248e-01,
          6.5116167e-02,  7.0884764e-02,  2.7545333e-01, -6.0635167e-01,
         -2.1278188e-01,  4.15110

In [7]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [8]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [9]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


In [10]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [11]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [12]:
block.layers

In [13]:
len(block.variables)

18

In [14]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

In [15]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [16]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_3 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_4 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________